In [2]:
import requests
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import geopy.geocoders as geocoders
import pandas as pd
import geocoder
from random import randint
import random
import time
import os
from unidecode import unidecode
from fuzzywuzzy import process

import os
import zipfile
import requests
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import geopy.geocoders as geocoders
import pandas as pd
import geocoder
from random import randint
import random
import time
import os
import fiona
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

c:\Users\Administrador\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
csv_files = os.listdir("2022_12\Registros_1_2\DEPARTAMENTOS_REG_1")


In [10]:
# Initialize an empty DataFrame
df_combined = []

# Iterate over the CSV files
for file in csv_files:
    # Read the CSV file
    df = pd.read_csv("2022_12/Registros_1_2/DEPARTAMENTOS_REG_1/" + str(file), encoding='latin-1')
    
    # Extract the 'DEPARTAMENTO' and 'MUNICIPIO' columns
    extracted_columns = df[['DEPARTAMENTO', 'MUNICIPIO']].drop_duplicates()
    
    # Append the extracted columns to the combined DataFrame
    df_combined.append(extracted_columns)

# Drop duplicates from the combined DataFrame
#df_combined = df_combined.drop_duplicates()

# Print the resulting DataFrame
#print(df_combined)

In [15]:
ofertas = pd.read_excel("24052023_Consolidado_Avaluos.xlsx", "Hoja1")

In [26]:
ofertas["Municipio"] = ofertas["Municipio"].str.strip().str.upper().astype(str).apply(lambda x: unidecode(x))
ofertas["Departamento"] = ofertas["Departamento"].str.strip().str.upper().astype(str).apply(lambda x: unidecode(x))

ofertas['Departamento'] = ofertas['Departamento'].replace('^BOGOTA$', 'BOGOTA, D.C.', regex=True)
ofertas['Departamento'] = ofertas['Departamento'].replace('^VALLE$', 'VALLE DEL CAUCA', regex=True)
ofertas['Departamento'] = ofertas['Departamento'].replace('^NTIOQUIA$', 'ANTIOQUIA', regex=True)
ofertas['Departamento'] = ofertas['Departamento'].replace('^GUAJIRA$', 'RIOHACHA', regex=True)
ofertas['Departamento'] = ofertas['Departamento'].replace('^MAGADALENA$', 'MAGDALENA', regex=True)
ofertas['Departamento'] = ofertas['Departamento'].replace('^VALLE DE CAUCA$', 'VALLE DEL CAUCA', regex=True)
ofertas['Departamento'] = ofertas['Departamento'].replace('^BARRANQUILLA$', 'ATLANTICO', regex=True)
ofertas['Departamento'] = ofertas['Departamento'].replace('^SINCELEJO$', 'SUCRE', regex=True)
ofertas['Departamento'] = ofertas['Departamento'].replace('^SANTADER$', 'SANTANDER', regex=True)
ofertas['Departamento'] = ofertas['Departamento'].replace('^NORTE SANTANDER$', 'NORTE DE SANTANDER', regex=True)


In [189]:
divipola = pd.read_excel("CODIGOS_DIVIPOLA.xlsx", "Listado Vigentes")

divipola["Nombre Municipio"] = divipola["Nombre Municipio"].astype(str).str.strip().apply(lambda x: unidecode(x))
divipola["Nombre Departamento"] = divipola["Nombre Departamento"].astype(str).str.strip().apply(lambda x: unidecode(x))

divipola["Código Municipio"] = divipola["Código Municipio"].astype(int).apply(round)
divipola["Código Departamento"] = divipola["Código Departamento"].astype(int).apply(round)

In [153]:

ofertas_2 = pd.merge(ofertas, divipola[["Nombre Departamento", "Código Departamento"]].drop_duplicates(), left_on = "Departamento", right_on = "Nombre Departamento", how = "left")

ofertas_2.loc[(ofertas_2["Departamento"].str.contains("[a-zA-Z]", regex = True,na = False)), "Departamento"] = ofertas_2.loc[(ofertas_2["Departamento"].str.contains("[a-zA-Z]", regex = True,na = False)), "Código Departamento"]
#ofertas_2.drop(["Nombre Departamento", "Código Departamento"], axis = 1, inplace = True)

In [73]:
divipola[divipola["Código Departamento"] == 11]

,Código Departamento,Código Municipio,Código Centro Poblado,Nombre Departamento,Nombre Municipio,Nombre Centro Poblado,Tipo
926,11,11001,11001000,"BOGOTA, D.C.","BOGOTA, D.C.","BOGOTÁ, D.C.",CM
927,11,11001,11001002,"BOGOTA, D.C.","BOGOTA, D.C.",NAZARETH,IP
928,11,11001,11001005,"BOGOTA, D.C.","BOGOTA, D.C.",SAN JUAN DE SUMAPAZ,C
929,11,11001,11001007,"BOGOTA, D.C.","BOGOTA, D.C.",PASQUILLA,CAS
930,11,11001,11001008,"BOGOTA, D.C.","BOGOTA, D.C.",SAN JUAN,C
931,11,11001,11001009,"BOGOTA, D.C.","BOGOTA, D.C.",BETANIA,CP
932,11,11001,11001010,"BOGOTA, D.C.","BOGOTA, D.C.",LA UNIÓN,CP


In [155]:
ofertas_2["Código Departamento"].fillna(9999, inplace=True)
divipola["Código Departamento"].fillna(9999, inplace=True)
ofertas_2["Código Departamento"] = ofertas_2["Código Departamento"].astype(int)
ofertas_2["Municipio"] = ofertas_2["Municipio"].astype(str)
divipola["Código Municipio"] = divipola["Código Municipio"].astype(int)
divipola["Código Departamento"] = divipola["Código Departamento"].astype(int)
divipola["Nombre Municipio"] = divipola["Nombre Municipio"].astype(str)

ofertas_2.loc[(ofertas_2["Municipio"].astype(str) == "1000") & (ofertas_2["DIVIPOLA"].astype(str).str.contains("[a-zA-Z]", regex = True,na = False)), "DIVIPOLA"] = ofertas_2["Departamento"].astype(str) + "001"
ofertas_2.loc[(ofertas_2["DIVIPOLA"].astype(str).str.contains("[a-zA-Z]", regex = True,na = False)) & (ofertas_2["Municipio"].astype(str).str.contains("000")), "DIVIPOLA"] = ofertas_2["Departamento"].astype(str) + ofertas_2["Municipio"].astype(str).str.replace("000", "")

#ofertas_2["Municipio"] = ofertas_2["Municipio"].str.replace("CALDAS|SANTANDER|\(VALLE\)|ANTIOQUIA|RISARALDA|SANTANDE|RISARALD|CORDOBA|MAGDALENA|BOYACA|CESAR|BOYACA|META|CAUCA|ANDER|SANTAN|ANTI|CUNDINAMARCA|CUNDINAMARC|-|ANTIOQUIA|VALLE$|CUNDINAMAR|SANTANDER|TOLIMA|NARINO|ATLANTICO|RISARALDA|NORTE SANT|BOLIVAR|QUINDIO|ATLANT|HUILA|CASANARE", "", regex = True).astype(str).str.strip()
#ofertas_2["Municipio"] = ofertas_2["Municipio"].str.replace("BOGOTA", "BOGOTA, D.C.")
#ofertas_2.loc[(ofertas_2["Municipio"].isin(["BOGOTA, D.C. D.C.", "BOGOTA, D.C."])) & (ofertas_2["Código Departamento"].isin([25, "25", "25.0", "25."])), "Municipio"] = 11001
#ofertas_2["Municipio"] = ofertas_2["Municipio"].str.replace("D.C. D.C.", "D.C.")
#ofertas_2.loc[(ofertas_2["Municipio"].isin(["BOGOTA, D.C. D.C.", "BOGOTA, D.C."])) & (ofertas_2["Código Departamento"].astype(int) == 25), "Código Departamento"] = 11 
#BOGOTA, D.C., D.C.
#ofertas_3 = pd.merge(ofertas_2, divipola[["Código Departamento", "Nombre Municipio", "Código Municipio"]].drop_duplicates(), how = "left", left_on = ["Código Departamento", "Municipio"], right_on = ["Código Departamento", "Nombre Municipio"])


In [198]:
df_municipios_IGAC = pd.concat(df_combined, axis = 0).drop_duplicates()
ofertas_2["DIVIPOLA"].fillna(0, inplace=True)
ofertas_2["DIVIPOLA"] = ofertas_2["DIVIPOLA"].astype(int)
df_municipios_IGAC["MUNICIPIO"] = df_municipios_IGAC["MUNICIPIO"].astype(int)

ofertas_3 = pd.merge(ofertas_2, df_municipios_IGAC, how = "inner", left_on = ["DIVIPOLA"], right_on = ["MUNICIPIO"])
print(ofertas_2.shape)
print(ofertas_3.shape)

(503493, 31)
(95736, 33)


In [199]:
ofertas_2["DIVIPOLA"].value_counts()

DIVIPOLA
11001    118586
25754     39539
76001     22694
8001      18662
5001      16854
          ...  
8558          1
25653         1
19785         1
19392         1
25896         1
Name: count, Length: 1058, dtype: int64

In [110]:
ofertas_3.loc[(ofertas_3["Municipio"].str.contains("[a-zA-Z]", regex = True,na = False)) & (ofertas_3["Código Municipio"].notnull())
              , "Municipio"] = ofertas_3.loc[(ofertas_3["Municipio"].str.contains("[a-zA-Z]", regex = True,na = False))  & (ofertas_3["Código Municipio"].notnull())
                                             , "Código Municipio"]

In [51]:
ofertas_3.head()

,ID,Fuente,Anio,Mes,Departamento,Municipio,DIVIPOLA,Municipio_nombre,Departamento_nombre,gestorIGAC,...,Deposito_Privado_Uso_Exclusivo,Anio_Construccion,Avaluo_Terreno,Avaluo_Construccion,Avaluo_Comercial,Unnamed: 28,Nombre Departamento,Código Departamento,Nombre Municipio,Código Municipio
0,1,A,2022,5,11,11001,11001.0,"BOGOTÁ, D.C.","BOGOTÁ, D.C.",No,...,NaN,NaN,0,47572703.48,2.661421e+07,NaN,NaN,9999,NaN,NaN
1,2,A,2022,5,11,11001,11001.0,"BOGOTÁ, D.C.","BOGOTÁ, D.C.",No,...,NaN,NaN,55871424,110674166.4,1.665456e+08,NaN,NaN,9999,NaN,NaN
2,3,A,2022,9,11,11001,11001.0,"BOGOTÁ, D.C.","BOGOTÁ, D.C.",No,...,NaN,NaN,65759384.59,149287094.07,2.150465e+08,NaN,NaN,9999,NaN,NaN
3,4,A,2022,10,41,41001,41001.0,NEIVA,HUILA,No,...,NaN,NaN,75052606.71,54495314.25,1.295479e+08,NaN,NaN,9999,NaN,NaN
4,5,A,2022,10,25,25473,25473.0,MOSQUERA,CUNDINAMARCA,Si,...,NaN,NaN,19512414.64,262767183.72,2.822796e+08,NaN,NaN,9999,NaN,NaN


In [185]:




ofertas_2[ofertas_2["DIVIPOLA"].astype(str).str.contains("[a-zA-Z]", regex = True,na = False)][["Municipio", "Departamento"]].value_counts()


 

Municipio              Departamento
212006                 05              32
109037                 76              16
307011                 68              15
622005                 76              10
418020                 19               3
520016                 76               3
308011                 76               3
1                      1                2
206013                 54               1
233001                 54               1
CIUDAD - CUNDINAMARCA  25.0             1
ANTIOQUIA - ANTIOQUIA  5.0              1
824005                 19               1
698011                 19               1
1012                   76               1
570013                 41               1
555015                 47               1
1019                   66               1
498021                 73               1
467001                 05               1
458009                 15               1
1003                   05               1
126011                 76               

In [149]:
df_municipios_IGAC = pd.concat(df_combined, axis = 0).drop_duplicates()
df_municipios_IGAC["DEPARTAMENTO"] = df_municipios_IGAC["DEPARTAMENTO"].astype(int)
df_municipios_IGAC["MUNICIPIO"] = df_municipios_IGAC["MUNICIPIO"].astype(int)

ofertas_4 = ofertas_3[(~ofertas_3["Municipio"].str.contains("[a-zA-Z]", regex = True,na = False)) & (~ofertas_3["Departamento"].str.contains("[a-zA-Z]", regex = True,na = False)) &
                      (ofertas_3["Municipio"].notnull()) & (ofertas_3["Departamento"].notnull())]

ofertas_4[ofertas_4["Municipio"].astype(str).str.replace(".0", "").str.replace(".", "").isnull()]

ofertas_4["Municipio"] = ofertas_4["Municipio"].astype(str).str.replace(".0", "").str.replace(".", "")
#ofertas_4["Municipio"] = ofertas_4["Municipio"].str.replace("", "99").astype(int)

ofertas_4["Departamento"] = ofertas_4["Departamento"].astype(str).str.replace(".0", "").str.replace(".", "").astype(int)
# 
ofertas_5 = ofertas_4[(ofertas_4["Municipio"].notnull()) & (ofertas_4["Departamento"].notnull()) & (ofertas_4["Departamento"] != "") & (ofertas_4["Municipio"] != "")]
ofertas_5["Municipio"] = ofertas_5["Municipio"].astype(int) 


#ofertas_3.shape

C:\Users\Administrador\AppData\Local\Temp\ipykernel_6820\1336454015.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ofertas_4["Municipio"] = ofertas_4["Municipio"].astype(str).str.replace(".0", "").str.replace(".", "")
C:\Users\Administrador\AppData\Local\Temp\ipykernel_6820\1336454015.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ofertas_4["Departamento"] = ofertas_4["Departamento"].astype(str).str.replace(".0", "").str.replace(".", "").astype(int)
C:\Users\Administrador\AppData\Local\Temp\ipyk

In [150]:
ofertas_6 = pd.merge(ofertas_5, df_municipios_IGAC, how = "left", left_on = ["Departamento", "Municipio"], right_on = ["DEPARTAMENTO", "MUNICIPIO"])

In [152]:
ofertas_6[ofertas_6["DEPARTAMENTO"].isnull()]["Municipio"].value_counts()

Municipio
1        226352
754       37220
1000      11798
430       10008
11001      9345
          ...  
397           1
29            1
247           1
94            1
896           1
Name: count, Length: 926, dtype: int64